To BE



In [1]:
!pip install -qq whatthelang
!pip3 install -qq twint

     |████████████████████████████████| 786 kB 18.1 MB/s 
     |████████████████████████████████| 970 kB 51.8 MB/s 
     |████████████████████████████████| 244 kB 71.9 MB/s 
     |████████████████████████████████| 265 kB 46.4 MB/s 
     |████████████████████████████████| 385 kB 52.4 MB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
     |████████████████████████████████| 288 kB 70.3 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 59 kB 7.7 MB/s 
     |████████████████████████████████| 140 kB 79.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 


In [2]:
# Import Library
from typing_extensions import Concatenate
import twint

In [88]:
import os
from google.colab import output
!pip install twint
!pip uninstall twint -y
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint/twint/
#!sed -i 's/tweet_count = 100/tweet_count = 100 if not config.Limit else config.Limit/g' url.py
%cd ..
!pip3 install . -r requirements.txt
%cd ..
output.clear()

In [89]:
!pip install aiohttp==3.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
import twint

## Get tweets

In this example you can see, how I got you the lovely #OKBoomer data

In [91]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [120]:
# Config for counting:

cc = twint.Config()
cc.Store_object = True
cc.Pandas = True
cc.Lang = 'Fa'
cc.Search = '#كرج'
cc.Store_csv = False
#cc.Output = "Karaj.csv"
#cc.Resume = 'Karaj_resume.csv'
cc.Since = "2022-09-17"
cc.Until = "2022-09-24"
cc.Filter_retweets = False
cc.Retries_count = 10000
cc.Count = True
cc.Native_retweets = False
cc.Limit = 1000
cc.Stats = True
cc.Hide_output = True

In [106]:
c = twint.Config()

c.Search = '#womensmarch'
c.Since = '2017-01-01'
c.Until = '2017-01-02'
c.Hide_output = True
c.Store_json = True
c.Output = 'womensmarch_2017.json'
c.Resume = 'wm_last.csv'
c.Debug = True
c.Count = True

In [127]:
# Run search
twint.run.Search(cc)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 0 Tweets.


In [116]:
# Quick check
twint.storage.panda.Tweets_df.head()

""


### The End
the stuff below is just some cleanup...

In [ ]:
# Cleanup
tweets = twint.storage.panda.Tweets_df.drop_duplicates(subset=['id'])

In [ ]:
# Reindex
tweets.index = range(len(tweets))

In [ ]:
# Remove non-english
from whatthelang import WhatTheLang
wtl = WhatTheLang()

In [ ]:
# This function makes easy to handle exceptions (e.g. no text where text should be)
# not really needed but can be useful 

def detect_lang(text):
    try: 
        return wtl.predict_lang(text)
    except Exception:
        return 'exp'

In [ ]:
# Added performance measure here...you can leave teh %%time line out

%%time

tweets['lang'] = tweets['tweet'].map(lambda t: detect_lang(t))

CPU times: user 30.2 ms, sys: 743 µs, total: 31 ms
Wall time: 32.3 ms


In [ ]:
# keep only english

tweets = tweets[tweets.lang == 'fa']

In [ ]:
# Done

tweets.to_json('tweets_boomer.json')

## Get peoples' connections

This is a short analysis in which I combine (very) basic scraping with extraction of Twitter networks and network analysis. 
The purpose was to identify interesting people on Twitter for me to follow...

The appropach:

- Fet links to all shows
- Fetch links to twitter-accounts form the shownotes
- Use these URLs to identify users
- Scrape all people these people follow

Assumption: People that are followed by people that are invited on TwimlAI are people, I should be following...



In [ ]:
# Import libraries
import re
import pickle # pickle is for storing element...pickling... you can store any kind of python object with that
import requests as rq

In [ ]:
# Load HTML parser library...yes, that's its name.
from bs4 import BeautifulSoup

In [ ]:
# Get URLs of all TWIML shows
r = rq.get('https://twimlai.com/shows/')

In [ ]:
# Parse the HTML
soup = BeautifulSoup(r.text)

In [ ]:
# Fetch all links from parsed HTML
links = soup.find_all('a')

In [ ]:
# Keep only links leading to a twiml-podcast
links = [l.attrs['href'] for l in links if l.attrs['href'].startswith('https://twimlai.com/twiml-talk')]

In [ ]:
# Drop duplicated links
links = list(set(links))

In [ ]:
# Iterate and fetch show-notes, then extract links leading to twitter. 
twitter_urls = []
for link in links:
  show = rq.get(link) # get shownotes 
  soup = BeautifulSoup(show.text) # parse
  show_links = soup.find_all('a') # find links 
  show_links = [l.attrs['href'] for l in show_links if l.attrs['href'].startswith('https://twitter.com')] # keep only links to twitter
  twitter_urls.extend(show_links) # store

In [ ]:
# Store the lovely list of links to twitter profiles
pickle.dump(list(set(twitter_urls)), open('twitter-list.p','wb'))

In [ ]:
# Unless already imported
import twint
import numpy as np

In [ ]:
# Filter out tooooo long twitter links that are more than likely not profiles
usernames = [x.replace('https://twitter.com/','') for x in set(twitter_urls) if len(x) <= 50]

In [ ]:
# Profile lookup

for username in usernames:
  c = twint.Config()
  c.Username = username
  c.Store_object = True
  c.User_full = False
  c.Pandas =True
  twint.run.Lookup(c)

In [ ]:
# Store in a DF
user_df = twint.storage.panda.User_df.drop_duplicates(subset=['id'])

In [ ]:
#Store away
user_df.to_csv('user_df.csv')

In [ ]:
# Or like that
user_df[['bio','username']].to_csv('short.csv')

In [ ]:
# Clean up
twint.storage.panda.clean()
twint.output.clean_follow_list()

In [ ]:
# Connect Google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# we iterate over the different usernames and store follower dataframe
for u in user_df['username']:
  c = twint.Config()
  c.Username = u
  c.Store_object = True
  c.User_full = False
  c.Pandas = True
  c.Store_pandas = True
  c.Stats = False
  c.Hide_output = True

  twint.run.Following(c)
  twint.storage.panda.Follow_df.to_pickle("/content/drive/My Drive/Colab/TWIML-guests/{}.p".format(u))
  twint.storage.panda.clean()
  twint.output.clean_follow_list()

In [ ]:
# To get the data back we use glob...which will help us dealing with many tiny filed
import glob

In [ ]:
# Get paths of all stored files
paths = glob.glob('/content/drive/My Drive/Colab/TWIML-guests/*.*')

In [ ]:
# Create an edgelist
# read stored DFs with following, append into long edgelist

empty = []
edgelist = pd.DataFrame(columns = ['target', 'source'])
for path in paths:
  df = pd.read_pickle(path)
  if len(df) == 1:
    name = df.index[0]
    edges = pd.DataFrame(df['following'][name], columns=['target'])
    edges['source'] = name
    edgelist = edgelist.append(edges)
  else:
    empty.append(path)

In [ ]:
# Reindex

edgelist.index = range(edgelist.shape[0])

### From here: Network analysis 101

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite 

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_edges_from([(u,v) for (u,v) in zip(edgelist['source'],edgelist['target'])])

In [ ]:
len(G.nodes)

87738

In [ ]:
eigenvector = nx.eigenvector_centrality(G)

In [ ]:
nx.set_node_attributes(G, eigenvector, 'eigenvector_centrality')

In [ ]:
import community

In [ ]:
G_und = G.to_undirected()

In [ ]:
communities = community.best_partition(G_und, resolution = 1)
nx.set_node_attributes(G, communities, 'community')

In [ ]:
perc_filter = np.percentile([v for u,v in eigenvector.items()], 90)

In [ ]:
nodes_selected = [x for x,y in eigenvector.items() if y >= perc_filter]

G_sub = G.subgraph(nodes_selected)

In [ ]:
communities = community.best_partition(G_sub.to_undirected(), resolution = 1)
nx.set_node_attributes(G_sub, communities, 'community_2')

In [ ]:
len(G_sub.nodes)

6772

In [ ]:
nx.write_gexf(G_sub, 'twiml.gexf')

In [ ]:
net_df = pd.DataFrame(dict(G_sub.nodes(data=True))).T

In [ ]:
net_df.groupby('community_2').apply(lambda t: t.sort_values(['eigenvector_centrality'],ascending=False)[:10])

In [ ]:
nlp_ppl = net_df[net_df.community_2 == 3].sort_values(['eigenvector_centrality'],ascending=False).index